# Lableme2YOLO

将使用Labeleme标注的数据集转为YOLO格式，并基于YOLOv8进行训练

## 下载Labelme样例数据集

In [4]:
# 下载数据集

!rm -rf SJB_25_Dataset.zip SJB_25_Dataset
!wget https://zihao-download.obs.cn-east-3.myhuaweicloud.com/yolov8/datasets/SJB_25_Dataset.zip

--2023-08-11 09:04:43--  https://zihao-download.obs.cn-east-3.myhuaweicloud.com/yolov8/datasets/SJB_25_Dataset.zip
Resolving zihao-download.obs.cn-east-3.myhuaweicloud.com (zihao-download.obs.cn-east-3.myhuaweicloud.com)... 121.36.235.162, 121.36.235.163
Connecting to zihao-download.obs.cn-east-3.myhuaweicloud.com (zihao-download.obs.cn-east-3.myhuaweicloud.com)|121.36.235.162|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80856957 (77M) [application/zip]
Saving to: ‘SJB_25_Dataset.zip’

SJB_25_Dataset.zip  100%[===================>]  77.11M  13.9MB/s    in 7.2s    

2023-08-11 09:04:54 (10.7 MB/s) - ‘SJB_25_Dataset.zip’ saved [80856957/80856957]



In [5]:
# 解压
!unzip SJB_25_Dataset.zip >> /dev/null

# 删除压缩包
!rm -rf SJB_25_Dataset.zip

## 划分数据集

In [6]:
import os
import shutil
import random
from tqdm import tqdm

In [7]:
Dataset_root = 'SJB_25_Dataset'

os.chdir(os.path.join(Dataset_root, 'images')) # os.chdir() 方法用于改变当前工作目录到指定的路径。

os.listdir()

['DSC_0285.jpg',
 'DSC_0274.jpg',
 'DSC_0293.jpg',
 'DSC_0260.jpg',
 'MVIMG_20230331_080914.jpg',
 'DSC_0245.jpg',
 'MVIMG_20230331_080912.jpg',
 'DSC_0265.jpg',
 'DSC_0280.jpg',
 'MVIMG_20230331_080915.jpg',
 'MVIMG_20230331_080920.jpg',
 'DSC_0278.jpg',
 'DSC_0289.jpg',
 'DSC_0259.jpg',
 'DSC_0301.jpg',
 'DSC_0240.jpg',
 'DSC_0209.jpg',
 'DSC_0219.jpg',
 'DSC_0284.jpg',
 'DSC_0283.jpg',
 'DSC_0269.jpg',
 'DSC_0236.jpg',
 'DSC_0281.jpg',
 'DSC_0282.jpg',
 'MVIMG_20230331_080908.jpg']

In [8]:
test_frac = 0.3  # 测试集比例，一般为三七开
random.seed(2) # 随机数种子，便于复现

In [9]:
folder = '.'

In [10]:
os.listdir(folder) # 由于已经通过os.chdir()切换到了‘images'目录，所以只需用‘.'即可索引其目录下的文件

['DSC_0285.jpg',
 'DSC_0274.jpg',
 'DSC_0293.jpg',
 'DSC_0260.jpg',
 'MVIMG_20230331_080914.jpg',
 'DSC_0245.jpg',
 'MVIMG_20230331_080912.jpg',
 'DSC_0265.jpg',
 'DSC_0280.jpg',
 'MVIMG_20230331_080915.jpg',
 'MVIMG_20230331_080920.jpg',
 'DSC_0278.jpg',
 'DSC_0289.jpg',
 'DSC_0259.jpg',
 'DSC_0301.jpg',
 'DSC_0240.jpg',
 'DSC_0209.jpg',
 'DSC_0219.jpg',
 'DSC_0284.jpg',
 'DSC_0283.jpg',
 'DSC_0269.jpg',
 'DSC_0236.jpg',
 'DSC_0281.jpg',
 'DSC_0282.jpg',
 'MVIMG_20230331_080908.jpg']

In [11]:
img_paths = os.listdir(folder) # os.listdir()用于返回一个由文件名和目录名组成的列表
random.shuffle(img_paths) # 随机排序

val_number = int(len(img_paths) * test_frac) # 测试集文件个数
train_files = img_paths[val_number:]         # 训练集文件名列表
val_files = img_paths[:val_number]           # 测试集文件名列表

print('数据集文件总数', len(img_paths))
print('训练集文件个数', len(train_files))
print('测试集文件个数', len(val_files))

数据集文件总数 25
训练集文件个数 18
测试集文件个数 7


## images文件夹整理

In [12]:
os.mkdir('train')
for each in tqdm(train_files):
    shutil.move(each, 'train')

100%|██████████| 18/18 [00:00<00:00, 19988.74it/s]


In [13]:
os.mkdir('val')
for each in tqdm(val_files):
    shutil.move(each, 'val')

100%|██████████| 7/7 [00:00<00:00, 15283.77it/s]


## labelme_jsons文件夹整理

In [14]:
os.chdir('../labelme_jsons')

In [15]:
os.mkdir('train')
for each in tqdm(train_files):
    srt_path = each.split('.')[0] + '.json'
    shutil.move(srt_path, 'train')

100%|██████████| 18/18 [00:00<00:00, 16669.79it/s]


In [16]:
os.mkdir('val')
for each in tqdm(val_files):
    srt_path = each.split('.')[0] + '.json'
    shutil.move(srt_path, 'val')

100%|██████████| 7/7 [00:00<00:00, 12968.25it/s]


In [17]:
os.chdir('../../')

## 单文件--Lableme2YOLO

In [18]:
import os
import json
import numpy as np

In [19]:
# 框的类别

bbox_class = {
    'sjb_rect':0
}

In [20]:
# 关键点的类别

keypoint_class = ['angle_30', 'angle_60', 'angle_90']

### 载入一个json文件

In [21]:
labelme_path = 'DSC_0281.json'

In [23]:
with open(labelme_path, 'r', encoding='utf-8') as f:
    labelme = json.load(f)

In [24]:
labelme.keys()

dict_keys(['version', 'flags', 'shapes', 'imagePath', 'imageData', 'imageHeight', 'imageWidth'])

In [25]:
labelme['version']

'5.1.1'

In [26]:
labelme['flags']

{}

In [27]:
# 图像文件名
labelme['imagePath']

'DSC_0281.jpg'

In [28]:
labelme['imageData']

In [29]:
# 图像高度
labelme['imageHeight']

3712

In [30]:
# 图像宽度
labelme['imageWidth']

5568

In [31]:
labelme['shapes']

[{'label': 'sjb_rect',
  'points': [[1138.0, 0.0], [4480.0, 1562.0]],
  'group_id': None,
  'shape_type': 'rectangle',
  'flags': {}},
 {'label': 'sjb_rect',
  'points': [[1160.0, 1954.0], [4560.0, 3566.0]],
  'group_id': None,
  'shape_type': 'rectangle',
  'flags': {}},
 {'label': 'angle_30',
  'points': [[4434.0, 284.0]],
  'group_id': None,
  'shape_type': 'point',
  'flags': {}},
 {'label': 'angle_30',
  'points': [[4526.0, 2316.0]],
  'group_id': None,
  'shape_type': 'point',
  'flags': {}},
 {'label': 'angle_60',
  'points': [[1202.0, 44.0]],
  'group_id': None,
  'shape_type': 'point',
  'flags': {}},
 {'label': 'angle_60',
  'points': [[1218.0, 2000.0]],
  'group_id': None,
  'shape_type': 'point',
  'flags': {}},
 {'label': 'angle_90',
  'points': [[1934.0, 1512.0]],
  'group_id': None,
  'shape_type': 'point',
  'flags': {}},
 {'label': 'angle_90',
  'points': [[1906.0, 3532.0]],
  'group_id': None,
  'shape_type': 'point',
  'flags': {}},
 {'label': 'sjb_poly',
  'points':

### 生成YOLO格式文件

In [32]:
img_width = labelme['imageWidth']   # 图像宽度
img_height = labelme['imageHeight'] # 图像高度

# 生成 YOLO 格式的 txt 文件
suffix = labelme_path.split('.')[-2]
yolo_txt_path = suffix + '.txt'

In [33]:
with open(yolo_txt_path, 'w', encoding='utf-8') as f:

    for each_ann in labelme['shapes']: # 遍历每个标注

        if each_ann['shape_type'] == 'rectangle': # 关键点信息的获取基于框的标注情况

            yolo_str = '' # 表示需要写入的信息

            ## 框的信息
            # 框的类别 ID
            bbox_class_id = bbox_class[each_ann['label']]
            yolo_str += '{} '.format(bbox_class_id)


            # 左上角和右下角的 XY 像素坐标
            bbox_top_left_x = int(min(each_ann['points'][0][0], each_ann['points'][1][0]))
            bbox_bottom_right_x = int(max(each_ann['points'][0][0], each_ann['points'][1][0]))
            bbox_top_left_y = int(min(each_ann['points'][0][1], each_ann['points'][1][1]))
            bbox_bottom_right_y = int(max(each_ann['points'][0][1], each_ann['points'][1][1]))


            # 框中心点的 XY 像素坐标
            bbox_center_x = int((bbox_top_left_x + bbox_bottom_right_x) / 2)
            bbox_center_y = int((bbox_top_left_y + bbox_bottom_right_y) / 2)


            # 框宽度
            bbox_width = bbox_bottom_right_x - bbox_top_left_x


            # 框高度
            bbox_height = bbox_bottom_right_y - bbox_top_left_y


            # 框中心点归一化坐标
            bbox_center_x_norm = bbox_center_x / img_width
            bbox_center_y_norm = bbox_center_y / img_height


            # 框归一化宽度
            bbox_width_norm = bbox_width / img_width


            # 框归一化高度
            bbox_height_norm = bbox_height / img_height


            yolo_str += '{:.5f} {:.5f} {:.5f} {:.5f} '.format(bbox_center_x_norm, bbox_center_y_norm, bbox_width_norm, bbox_height_norm)


            ## 找到该框中所有关键点，存在字典 bbox_keypoints_dict 中
            bbox_keypoints_dict = {}
            for each_ann in labelme['shapes']: # 遍历所有标注
                if each_ann['shape_type'] == 'point': # 筛选出关键点标注
                    # 关键点XY坐标、类别
                    x = int(each_ann['points'][0][0])
                    y = int(each_ann['points'][0][1])
                    label = each_ann['label']
                    if (x>bbox_top_left_x) & (x<bbox_bottom_right_x) & (y<bbox_bottom_right_y) & (y>bbox_top_left_y): # 筛选出在框中的关键点
                        bbox_keypoints_dict[label] = [x, y]


            ## 把关键点按顺序排好，具体顺序参照 keypoint_class
            for each_class in keypoint_class: # 遍历每一类关键点
                if each_class in bbox_keypoints_dict:
                    keypoint_x_norm = bbox_keypoints_dict[each_class][0] / img_width
                    keypoint_y_norm = bbox_keypoints_dict[each_class][1] / img_height


                    yolo_str += '{:.5f} {:.5f} {} '.format(keypoint_x_norm, keypoint_y_norm, 2) # 2-可见不遮挡 1-遮挡 0-没有点
                else: # 不存在的点，一律为0
                    yolo_str += '0 0 0 '.format(keypoint_x_norm, keypoint_y_norm, 0)


            # 写入 txt 文件中
            f.write(yolo_str + '\n')

print('{} --> {} 转换完成'.format(labelme_path, yolo_txt_path))

DSC_0281.json --> DSC_0281.txt 转换完成


## 批量--Lableme2YOLO

In [34]:
import os
import json
import shutil
import numpy as np
from tqdm import tqdm

In [35]:
Dataset_root = 'SJB_25_Dataset'

In [36]:
# 指定类别信息

# 框的类别
bbox_class = {
    'sjb_rect':0
}

# 关键点的类别
keypoint_class = ['angle_30', 'angle_60', 'angle_90']

In [37]:
os.chdir(Dataset_root)

In [38]:
# 重新创建一个labels的文件夹用于保存txt文件

os.mkdir('labels')
os.mkdir('labels/train')
os.mkdir('labels/val')

In [39]:
# 之前已成功完成对单一文件json->txt格式的转换，接下来将其封装为函数，进行批量处理

def process_single_json(labelme_path, save_folder='../../labels/train'):

    with open(labelme_path, 'r', encoding='utf-8') as f:
        labelme = json.load(f)

    img_width = labelme['imageWidth']   # 图像宽度
    img_height = labelme['imageHeight'] # 图像高度

    # 生成 YOLO 格式的 txt 文件
    suffix = labelme_path.split('.')[-2]
    yolo_txt_path = suffix + '.txt'

    with open(yolo_txt_path, 'w', encoding='utf-8') as f:

        for each_ann in labelme['shapes']: # 遍历每个标注

            if each_ann['shape_type'] == 'rectangle': # 每个框，在 txt 里写一行

                yolo_str = ''

                ## 框的信息
                # 框的类别 ID
                bbox_class_id = bbox_class[each_ann['label']]
                yolo_str += '{} '.format(bbox_class_id)
                # 左上角和右下角的 XY 像素坐标
                bbox_top_left_x = int(min(each_ann['points'][0][0], each_ann['points'][1][0]))
                bbox_bottom_right_x = int(max(each_ann['points'][0][0], each_ann['points'][1][0]))
                bbox_top_left_y = int(min(each_ann['points'][0][1], each_ann['points'][1][1]))
                bbox_bottom_right_y = int(max(each_ann['points'][0][1], each_ann['points'][1][1]))
                # 框中心点的 XY 像素坐标
                bbox_center_x = int((bbox_top_left_x + bbox_bottom_right_x) / 2)
                bbox_center_y = int((bbox_top_left_y + bbox_bottom_right_y) / 2)
                # 框宽度
                bbox_width = bbox_bottom_right_x - bbox_top_left_x
                # 框高度
                bbox_height = bbox_bottom_right_y - bbox_top_left_y
                # 框中心点归一化坐标
                bbox_center_x_norm = bbox_center_x / img_width
                bbox_center_y_norm = bbox_center_y / img_height
                # 框归一化宽度
                bbox_width_norm = bbox_width / img_width
                # 框归一化高度
                bbox_height_norm = bbox_height / img_height

                yolo_str += '{:.5f} {:.5f} {:.5f} {:.5f} '.format(bbox_center_x_norm, bbox_center_y_norm, bbox_width_norm, bbox_height_norm)

                ## 找到该框中所有关键点，存在字典 bbox_keypoints_dict 中
                bbox_keypoints_dict = {}
                for each_ann in labelme['shapes']: # 遍历所有标注
                    if each_ann['shape_type'] == 'point': # 筛选出关键点标注
                        # 关键点XY坐标、类别
                        x = int(each_ann['points'][0][0])
                        y = int(each_ann['points'][0][1])
                        label = each_ann['label']
                        if (x>bbox_top_left_x) & (x<bbox_bottom_right_x) & (y<bbox_bottom_right_y) & (y>bbox_top_left_y): # 筛选出在该个体框中的关键点
                            bbox_keypoints_dict[label] = [x, y]

                ## 把关键点按顺序排好
                for each_class in keypoint_class: # 遍历每一类关键点
                    if each_class in bbox_keypoints_dict:
                        keypoint_x_norm = bbox_keypoints_dict[each_class][0] / img_width
                        keypoint_y_norm = bbox_keypoints_dict[each_class][1] / img_height
                        yolo_str += '{:.5f} {:.5f} {} '.format(keypoint_x_norm, keypoint_y_norm, 2) # 2-可见不遮挡 1-遮挡 0-没有点
                    else: # 不存在的点，一律为0
                        yolo_str += '0 0 0 '
                # 写入 txt 文件中
                f.write(yolo_str + '\n')

    shutil.move(yolo_txt_path, save_folder)
    print('{} --> {} 转换完成'.format(labelme_path, yolo_txt_path))

In [40]:
# 整理所得到的YOLO格式标注文件，使其与train，val的数据一一对应

#-------------------------------------训练集------------------------------------
os.chdir('labelme_jsons/train')

save_folder = '../../labels/train'
for labelme_path in os.listdir():
    process_single_json(labelme_path, save_folder=save_folder)
print('YOLO格式的txt标注文件已保存至 ', save_folder)

os.chdir('../../') # 初始化路径


#-------------------------------------验证集------------------------------------
os.chdir('labelme_jsons/val')

save_folder = '../../labels/val'
for labelme_path in os.listdir():
    process_single_json(labelme_path, save_folder=save_folder)
print('YOLO格式的txt标注文件已保存至 ', save_folder)

os.chdir('../../') # 初始化路径

DSC_0245.json --> DSC_0245.txt 转换完成
DSC_0301.json --> DSC_0301.txt 转换完成
MVIMG_20230331_080920.json --> MVIMG_20230331_080920.txt 转换完成
MVIMG_20230331_080908.json --> MVIMG_20230331_080908.txt 转换完成
DSC_0282.json --> DSC_0282.txt 转换完成
DSC_0240.json --> DSC_0240.txt 转换完成
DSC_0269.json --> DSC_0269.txt 转换完成
DSC_0236.json --> DSC_0236.txt 转换完成
DSC_0280.json --> DSC_0280.txt 转换完成
DSC_0265.json --> DSC_0265.txt 转换完成
DSC_0293.json --> DSC_0293.txt 转换完成
DSC_0284.json --> DSC_0284.txt 转换完成
DSC_0259.json --> DSC_0259.txt 转换完成
DSC_0219.json --> DSC_0219.txt 转换完成
MVIMG_20230331_080912.json --> MVIMG_20230331_080912.txt 转换完成
DSC_0274.json --> DSC_0274.txt 转换完成
MVIMG_20230331_080915.json --> MVIMG_20230331_080915.txt 转换完成
DSC_0278.json --> DSC_0278.txt 转换完成
YOLO格式的txt标注文件已保存至  ../../labels/train
DSC_0285.json --> DSC_0285.txt 转换完成
DSC_0281.json --> DSC_0281.txt 转换完成
DSC_0289.json --> DSC_0289.txt 转换完成
MVIMG_20230331_080914.json --> MVIMG_20230331_080914.txt 转换完成
DSC_0260.json --> DSC_0260.txt 转换完成
DSC